# Making Python Code Faster with Numba

In [3]:
from numba import jit
import random

def monte_carlo_pi(nSamples):
    acc=0
    for i in range(nSamples):
        x = random.random()
        y = random.random()
        if (x**2 + y**2) < 1.0:
            acc += 1
    return 4.0 * acc/nSamples

In [4]:
%time monte_carlo_pi(10000)

CPU times: total: 0 ns
Wall time: 3 ms


3.1724

In [5]:
monte_carlo_pi_jit = jit()(monte_carlo_pi)

C:\Users\anura\AppData\Local\Temp\ipykernel_3728\3627295421.py:1: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  monte_carlo_pi_jit = jit()(monte_carlo_pi)


In [6]:
%time monte_carlo_pi_jit(10000)

CPU times: total: 172 ms
Wall time: 1.2 s


3.1576

Time increases if you run it once with jit because of compilation time, but if you run it again it decreases.

## Another Example

In [7]:
from numba import jit, njit, vectorize

In [8]:
def original_func(input_list):
    output_list = []
    for item in input_list:
        if item%2 == 0:
            output_list.append(2)
        else:
            output_list.append('1')
    return output_list

test_array = list(range(100000))

In [9]:
%time _ = original_func(test_array)

CPU times: total: 0 ns
Wall time: 4 ms


In [10]:
jitted_func = jit()(original_func)

C:\Users\anura\AppData\Local\Temp\ipykernel_3728\868546817.py:1: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  jitted_func = jit()(original_func)


In [11]:
%time _ = jitted_func(test_array)

C:\Users\anura\AppData\Local\Temp\ipykernel_3728\2518354451.py:1: NumbaWarning: 
Compilation is falling back to object mode WITH looplifting enabled because Function "original_func" failed type inference due to: Invalid use of BoundFunction(list.append for list(int64)<iv=None>) with parameters (Literal[str](1))

During: resolving callee type: BoundFunction(list.append for list(int64)<iv=None>)
During: typing of call at C:\Users\anura\AppData\Local\Temp\ipykernel_3728\2518354451.py (7)


File "..\..\..\..\AppData\Local\Temp\ipykernel_3728\2518354451.py", line 7:
<source missing, REPL/exec in use?>

  def original_func(input_list):
C:\Users\anura\AppData\Local\Temp\ipykernel_3728\2518354451.py:1: NumbaWarning: 
Compilation is falling back to object mode WITHOUT looplifting enabled because Function "original_func" failed type inference due to: Cannot determine Numba type of <class 'numba.core.dispatcher.LiftedLoop'>

File "..\..\..\..\AppData\Local\Temp\ipykernel_3728\2518354451.py", line

CPU times: total: 62.5 ms
Wall time: 265 ms


Numba cannot tell whether to append number or a string.
```python
if item%2 == 0:
    output_list.append(2)
else:
    output_list.append('1')
```
```
Compilation is falling back to object mode WITH looplifting enabled because Function "original_func" failed type inference due to: Invalid use of BoundFunction(list.append for list(int64)<iv=None>) with parameters (Literal[str](1))

During: resolving callee type: BoundFunction(list.append for list(int64)<iv=None>)
```

In [12]:
%time _ = jitted_func(test_array)

CPU times: total: 0 ns
Wall time: 15 ms


In [13]:
jitted_func = jit(nopython=True)(original_func)

In [14]:
%time _ = jitted_func(test_array)

TypingError: Failed in nopython mode pipeline (step: nopython frontend)
[1m[1m[1mInvalid use of BoundFunction(list.append for list(int64)<iv=None>) with parameters (Literal[str](1))
[0m
[0m[1mDuring: resolving callee type: BoundFunction(list.append for list(int64)<iv=None>)[0m
[0m[1mDuring: typing of call at C:\Users\anura\AppData\Local\Temp\ipykernel_3728\2518354451.py (7)
[0m
[1m
File "..\..\..\..\AppData\Local\Temp\ipykernel_3728\2518354451.py", line 7:[0m
[1m<source missing, REPL/exec in use?>[0m


So now with ```nopython=True``` we get the error: ```TypingError: Failed in nopython mode pipeline (step: nopython frontend)
Invalid use of BoundFunction(list.append for list(int64)) with parameters (Literal[str](1))```

```nopython``` mode is a compilation mode that enforces the strictest form of type inference and optimization. When you decorate a function with ```@numba.jit(nopython=True)```, you are instructing Numba to compile the function in a way that avoids falling back to Python objects as much as possible, resulting in more efficient machine code.

### Now, we correct the function so that the types match

In [15]:
def original_func(input_list):
    output_list = []
    for item in input_list:
        if item%2 == 0:
            output_list.append(2)
        else:
            output_list.append(1)
    return output_list

test_array = list(range(100000))

In [16]:
jitted_func = njit()(original_func)

In [17]:
%time _ = jitted_func(test_array)

c:\Users\anura\AppData\Local\Programs\Python\Python310\lib\site-packages\numba\core\ir_utils.py:2149: NumbaPendingDeprecationWarning: 
Encountered the use of a type that is scheduled for deprecation: type 'reflected list' found for argument 'input_list' of function 'original_func'.

For more information visit https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-reflection-for-list-and-set-types

File "..\..\..\..\AppData\Local\Temp\ipykernel_3728\3611880893.py", line 1:
<source missing, REPL/exec in use?>

  warnings.warn(NumbaPendingDeprecationWarning(msg, loc=loc))


CPU times: total: 62.5 ms
Wall time: 217 ms


In [18]:
%time _ = jitted_func(test_array)

CPU times: total: 0 ns
Wall time: 90.5 ms


So, avoid using python lists with Numba. Because no optimization.

Numba provides specialized support for NumPy arrays and some array-like operations, but it doesn't provide direct support for regular Python lists. The reason for this is that Python lists are dynamic, heterogeneous, and involve more complex memory management compared to NumPy arrays. 

Let's use arrays

In [19]:
import numpy as np

def original_func(input_list):
    output_list = []
    for item in input_list:
        if item%2 == 0:
            output_list.append(2)
        else:
            output_list.append(1)
    return output_list

test_array = np.arange(100000)

In [20]:
jitted_func = njit()(original_func)

In [21]:
%time _ = jitted_func(test_array)

CPU times: total: 46.9 ms
Wall time: 107 ms
